# username: samuel_andrey
_notebook for model prediction testing_

In [62]:
import pandas as pd
import tensorflow as tf
import requests
import json
import base64

In [63]:
data = pd.read_csv('data/heart.csv')

ROW_TESTING = 1 # Change row testing from this

HEART_DISEASE_LABELS = {
    0: "Normal",
    1: "Heart Disease"
}

In [64]:
data_test = data.iloc[ROW_TESTING, :12].to_dict()
data_test

{'Age': 49,
 'Sex': 'F',
 'ChestPainType': 'NAP',
 'RestingBP': 160,
 'Cholesterol': 180,
 'FastingBS': 'No',
 'RestingECG': 'Normal',
 'MaxHR': 156,
 'ExerciseAngina': 'N',
 'Oldpeak': 1.0,
 'ST_Slope': 'Flat',
 'HeartDisease': 1}

In [65]:
heart_disease_value = data_test["HeartDisease"]
heart_disease_label = HEART_DISEASE_LABELS.get(heart_disease_value, "Unknown")
print(heart_disease_label)

Heart Disease


In [66]:
data_test = data.iloc[ROW_TESTING, :11].to_dict()
data_test

{'Age': 49,
 'Sex': 'F',
 'ChestPainType': 'NAP',
 'RestingBP': 160,
 'Cholesterol': 180,
 'FastingBS': 'No',
 'RestingECG': 'Normal',
 'MaxHR': 156,
 'ExerciseAngina': 'N',
 'Oldpeak': 1.0,
 'ST_Slope': 'Flat'}

In [67]:
def prepare_feature_spec(test_input: dict):
    feature_spec = {}
    for key, value in test_input.items():
        if isinstance(value, float):
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
    return feature_spec

def serialize_example(feature_spec):
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec))
    return example.SerializeToString()

def encode_example(example):
    return base64.b64encode(example).decode()

def prepare_data(test_input):
    feature_spec = prepare_feature_spec(test_input)
    serialized_example = serialize_example(feature_spec)
    encoded_example = encode_example(serialized_example)
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [{
            "examples": {"b64": encoded_example}
        }]
    })

In [68]:
json_data = prepare_data(data_test)

endpoint = "http://103.190.215.140:8501/v1/models/cc-model:predict"
response = requests.post(endpoint, data=json_data)

predictions = response.json().get("predictions")
result = "Normal" if predictions and predictions[0][0] < 0.5 else "Heart Disease" if predictions else "Error: No predictions found."

print("Prediction Result: " + result)
print("   Actually Label: " + heart_disease_label)

Prediction Result: Heart Disease
   Actually Label: Heart Disease
